In [22]:
# Import necessary modules and functions

import pandas as pd
import seaborn as sns
import numpy as np
import nltk
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

kaggledf = pd.read_csv('KS_train_data.csv', delimiter = ',')

In [23]:
test_columns = ["project_id", "blurb", "category", "country", "created_at", "currency", "deadline", "fx_rate", "goal", "launched_at", "name", "staff_pick", "location", "subcategory", "project_url"]

#drop colums that are not in test_set + reward URL
kaggledf = kaggledf.drop(columns=[col for col in kaggledf if col not in test_columns])

In [24]:
kaggledf["seconds"] = kaggledf["deadline"] - kaggledf["created_at"]

In [25]:
#convert all goals to USD
def convert_usd(row):
    return row["goal"] * row["fx_rate"]
    
    
kaggledf["usd_goal"] = kaggledf.apply(lambda row: convert_usd(row), axis = 1)


#delete unnessesary colums
currency_col = ["fx_rate", "currency", "goal"]
kaggledf = kaggledf.drop(columns=[col for col in kaggledf if col in currency_col])

In [30]:
#strip url of unnessesary stuff
def stripper(row): #XD
    raw = row["project_url"].split("/")[-1].split("-")
    return " ".join(raw)

kaggledf["stripped_url"] = kaggledf.apply(lambda row: stripper(row), axis = 1)

#delete original url column
kaggledf = kaggledf.drop(columns = ["project_url"])

In [31]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
#categorical ranges for goal and pledged, sentiment vader (numerical & categorical)  
#columns to be removed: url's, location, id,
kaggledf.dropna(inplace=True)
kaggledf

,project_id,blurb,category,country,created_at,deadline,launched_at,name,staff_pick,location,subcategory,seconds,usd_goal,stripped_url
0,KS_000000,I will be an artist-in-residence at Elsewhere ...,art,US,1325980505,1334866560,1332969260,Elsewhere Studios artist-in-residency program!,False,"Paonia, CO",mixed media,8886055,2800.000000,elsewhere studios artist in residency program
1,KS_000001,1000 Artists is a public art-making installati...,art,US,1330926084,1336440145,1332984145,1000 Artists: Presidential Inauguration 2013,False,"Washington, DC",public art,5514061,4500.000000,1000 artists presidential inauguration 2013
2,KS_000002,"The Sequel to ‘My Favorite Machine”, ""MyPhoneH...",art,US,1332382894,1338175739,1332991739,MyPhoneHenge,False,"Frisco, TX",sculpture,5792845,5000.000000,myphonehenge
3,KS_000003,A film exploring the role and value of art edu...,art,GB,1332342451,1334806096,1332991696,Walk-Through,False,"Glasgow, UK",art,2463645,6500.000000,walk through
4,KS_000004,We need to build a kitchen for Habitable Space...,art,US,1328562617,1335584240,1332992240,A kitchen for Habitable Spaces,False,"Kingsbury, TX",public art,7021623,15000.000000,a kitchen for habitable spaces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,KS_099995,Make the workshop/ small stage space at Jimmy'...,theater,US,1348677893,1356552252,1353960252,"Help Fund the ""Back Room"" Arts Space at Jimmy'...",False,"New York, NY",spaces,7874359,7000.000000,help fund the back room arts space at jimmys n...
99996,KS_099996,Help create a permanent home for live comedy s...,theater,US,1364939099,1367674009,1365082009,The Coalition Theater,False,"Richmond, VA",spaces,2734910,25000.000000,the coalition theater 0
99997,KS_099997,Voix de Ville is a pop-up imaginarium of neo-v...,theater,US,1366078694,1373738400,1370568560,The Voix De Ville! : A Pop-up Theater and Cabaret,True,"Cleveland, OH",spaces,7659706,5500.000000,the voix de ville a pop up theater and cabaret
99998,KS_099998,Help Synetic Theater create a new Studio to pr...,theater,US,1372177000,1376654340,1373568644,Save the Studio!,False,"Arlington, VA",spaces,4477340,25000.000000,save the studio
